# BIDS Conversion

Run heudiconv. Scripts stored in:
`/oleary/functional/UICL/BIDS/code`

details in **Heudiconv.ipynb**

Then, check to see if heudiconv failed for any subject using **check_missing_outputs.ipynb**

# Time 2 - Check slice timing

First for data from the Siemens

In [ ]:
%%bash

# if json file didn't include slice timing, echo subid

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_siemens.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_siemens.txt))

bidsdir=/Shared/oleary/functional/UICL/BIDS/rawdata

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ; do

json=$bidsdir/sub-${sub[$n]}/ses-${ses[$n]}/func/sub-${sub[$n]}_ses-${ses[$n]}_task-rest_bold.json

if ! grep -q "SliceTiming" $json ; then
    echo ${sub[$n]}_${ses[$n]}

fi

done

# 3011_63029716 USE GE

run the `get_slice_time.py` script to get the slice timing information for GE scanner.

```
usage: python /path/to/get_slice_time.py /path/to/dicom/pattern* /out/path

python /Shared/oleary/functional/UICL/BIDS/code/get_slice_time.py "/Shared/oleary/functional/UICL/dicomdata/3040/63910816/FMRI_004/63910816_004_*" "/Shared/oleary/functional/UICL/BIDS/code/time2/"

```

### then add slice timing infor for GE

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt))

bidsdir=/Shared/oleary/functional/UICL/BIDS/rawdata

for n in `seq -s " " 0 1 97` ; do
    json=$bidsdir/sub-${sub[$n]}/ses-${ses[$n]}/func/sub-${sub[$n]}_ses-${ses[$n]}_task-rest_bold.json
    slicetimetxt=/Shared/oleary/functional/UICL/BIDS/code/time2/SliceTiming.json

    if ! grep -q "SliceTiming" $json ; then
        echo ${sub[$n]}_${ses[$n]} "needs SliceTiming info, adding now"
        chmod 777 $json
        cat $slicetimetxt '\n' >> $json `# this paste things weiredly, have to fix with sed below`
    else
        echo "already had SliceTiming info, moving on"
    fi

    sed -i 's/'}{'/',\\n'/g' $json

done

# Check all 119 to see if anybody doesn't have slice time info

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

bidsdir=/Shared/oleary/functional/UICL/BIDS/rawdata

for n in `seq -s " " 0 1 118` ; do
    json=$bidsdir/sub-${sub[$n]}/ses-${ses[$n]}/func/sub-${sub[$n]}_ses-${ses[$n]}_task-rest_bold.json
    slicetimetxt=/Shared/oleary/functional/UICL/BIDS/code/time2/SliceTiming.json

    if ! grep -q "SliceTiming" $json ; then
        echo ${sub[$n]}_${ses[$n]} "needs SliceTiming info"
    else
        echo "already had SliceTiming info, moving on"
    fi

done

# Exclude dummy TRs

Once you get the data in BIDS format, you need to get rid of the dummy TRs at the beginning of the run BEFORE running FMRIPREP

Run **/oleary/functional/UICL/BIDS/code/exclude_first_TRs.sh**. For the resting state scanes in UICL, we will exclude the first 3 TRs (6 seconds)

Remember to make a note of this in a task json file (/oleary/functional/UICL/BIDS/task-rest_bold.json) in the following fields:

`"NumberOfVolumesDiscardedByUser": 3,
"NumberOfVolumesDiscardedByScanner": 2,`

Rest data = 177 volumes x 2s TR = 354s = 5.9 mins

**Time 2**

/Shared/oleary/functional/UICL/BIDS/code/exclude_first_TRs_time2.sh

# Check if the non-steady state timepoints have been removed for everybody

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

bidsdir=/Shared/oleary/functional/UICL/BIDS/rawdata

for n in `seq -s " " 0 1 118` ; do
    rest=$bidsdir/sub-${sub[$n]}/ses-${ses[$n]}/func/sub-${sub[$n]}_ses-${ses[$n]}_task-rest_bold.nii.gz
    x=$(fslnvols $rest)

    if [ $x  == 177 ] ; then
    echo "Non-steady state timepoints were already removed"
    fi
done

# test phase encoding direction

"PhaseEncodingDirection": "j-" but is AP

In [ ]:
# test with dcm2niix
# Siemens resting state time 1
/Shared/pinc/sharedopt/apps/dcm2niix/Linux/x86_64/1.0.20190902/dcm2niix \
-z y -f %s -o ./ \
/Shared/oleary/functional/UICL/dicomdata/3040/64069714/FMRI_003/

# GE resting state time 2
/Shared/pinc/sharedopt/apps/dcm2niix/Linux/x86_64/1.0.20190902/dcm2niix \
-z y -f %s -o ./ \
/Shared/oleary/functional/UICL/dicomdata/3040/63910816/FMRI_004/

In [ ]:
# test with Joel's script

source /raid0/homes/tientong/sourcefiles/afni_source.sh
source /raid0/homes/tientong/sourcefiles/fsl_source.sh
source /raid0/homes/tientong/sourcefiles/freesurfer_source.sh
source /raid0/homes/tientong/sourcefiles/gdcm_source.sh

bash /Shared/oleary/functional/UICL/BIDS/code/GE_dcm_to_nii.sh \
-i /Shared/oleary/functional/UICL/dicomdata/3040/63910816/FMRI_004/ \
-o /Shared/oleary/functional/UICL/BIDS/code -B -v

# PhaseEncodingDirection=AP

In [41]:
from nipype.utils.filemanip import loadpkl
path = "/Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3040_wf/func_preproc_ses_63910816_task_rest_wf/ica_aroma_wf/ica_aroma_confound_extraction/"
res = loadpkl(path + 'result_ica_aroma_confound_extraction.pklz')
res

{'in_file': '/mnt/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/merge/vol0000_xform-00000_merged.nii.gz',
 'mc_method': 'AFNI'}

# Run FMRIPREP

Note: As of Sep 5 2019 fmriprep version 1.4.1 the ANTs command for fieldmap-less distortion correction (`--use-syn-sdc`) didn't work with the new MNI152 template (details here https://github.com/poldracklab/fmriprep/issues/1665)

Therefore for UICL time 1 resting state data, this option was turned OFF.

In [ ]:
# pull the lastest version of fmriprep

singularity pull docker://poldracklab/fmriprep:<version number>
        
# First create a template file for submitting jobs on argon


#!/bin/sh
#$ -pe smp 10
#$ -q PINC,UI,CCOM
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/fmriprep/out/rest
#$ -e /Users/tientong/logs/uicl/fmriprep/err/rest
OMP_NUM_THREADS=8 
singularity run -H /Users/tientong/singularity_home \
-B /Shared/oleary/:/mnt \
/Users/tientong/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img \
/mnt/functional/UICL/BIDS \
/mnt/functional/UICL/BIDS/derivatives/fmriprep/rest \
participant --participant-label SUBJECT --skip_bids_validation \
-t rest --fs-license-file /mnt/functional/FreeSurferLicense/license.txt --fs-no-reconall \
-w /mnt/functional/UICL/BIDS/derivatives/fmriprep/rest --write-graph \
--use-aroma --error-on-aroma-warnings --output-spaces T1w func MNI152NLin6Asym:res-2 --stop-on-first-crash \
--omp-nthreads 8 --nthreads 8 --mem_mb 22500 --notrack

In [ ]:
%%bash

# then run this on argon

for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/subjects.txt | tr '\n' ' ') ; do
sed -e "s|SUBJECT|${sub}|" fmriprep_rest_TEMPLATE.job > fmriprep_rest_sub-${sub}.job
done

for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/subjects.txt | tr '\n' ' ') ; do
    nohup qsub fmriprep_rest_sub-${sub}.job
done 

# Time 2

template file

In [ ]:
%%bash

#!/bin/sh
#$ -pe smp 10
#$ -q PINC,UI,CCOM
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/fmriprep/out/rest_time2
#$ -e /Users/tientong/logs/uicl/fmriprep/err/rest_time2
OMP_NUM_THREADS=8
singularity run -H /Users/tientong/singularity_home \
-B /Shared/oleary/:/mnt \
/Users/tientong/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img \
/mnt/functional/UICL/BIDS \
/mnt/functional/UICL/BIDS/derivatives/fmriprep/rest \
participant --participant-label SUBJECT --skip_bids_validation \
-t rest --fs-license-file /mnt/functional/FreeSurferLicense/license.txt --fs-no-reconall \
-w /mnt/functional/UICL/BIDS/derivatives/fmriprep/rest --write-graph \
--use-aroma --error-on-aroma-warnings --output-spaces T1w MNI152NLin6Asym:res-2 --stop-on-first-crash \
--omp-nthreads 8 --nthreads 8 --mem_mb 22500 --notrack

In [ ]:
# then run this on argon

for sub in $(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt) ; do
sed -e "s|SUBJECT|${sub}|" fmriprep_rest_TEMPLATE.job > fmriprep_rest_sub-${sub}.job
done



for sub in  $(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_group2.txt) ; do
    nohup qsub fmriprep_rest_sub-${sub}.job
done 

# Run MRIQC

In [ ]:
# pull the lastest version of mriqc

singularity pull docker://poldracklab/mriqc:<version number>

# run the code below on Argon

#!/bin/sh
#$ -pe smp 10
#$ -q PINC,UI,CCOM
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/mriqc/out
#$ -e /Users/tientong/logs/uicl/mriqc/err

singularity run -H /Users/tientong/singularity_home \
/Users/tientong/mriqc_0.15.1.sif \
/Shared/oleary/functional/UICL/BIDS/rawdata \
/Shared/oleary/functional/UICL/BIDS/derivatives/mriqc \
participant --participant_label SUBJECT --no-sub -m T1w \
-w /Shared/oleary/functional/UICL/BIDS/derivatives/mriqc \
--verbose-reports --write-graph \
--n_procs 8 --mem_gb 22 --profile

In [ ]:
#!/bin/sh
#$ -pe smp 10
#$ -q PINC,UI,CCOM
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/mriqc/out
#$ -e /Users/tientong/logs/uicl/mriqc/err

singularity run -H /Users/tientong/singularity_home \
/Users/tientong/mriqc_0.15.1.sif \
/Shared/oleary/functional/UICL/BIDS/rawdata \
/Shared/oleary/functional/UICL/BIDS/derivatives/mriqc \
participant --participant_label SUBJECT --no-sub -m bold --task-id rest \
-w /Shared/oleary/functional/UICL/BIDS/derivatives/mriqc \
--verbose-reports --write-graph --hmc-fsl --correct-slice-timing \
--n_procs 8 --mem_gb 22 --profile

In [ ]:
%%bash

for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/allsub.txt | tr '\n' ' ') ; do
sed -e "s|SUBJECT|${sub}|g" mriqc_t1w_TEMPLATE.job > mriqc_t1w_${sub}.job
done


for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/allsub.txt | tr '\n' ' ') ; do
qsub mriqc_t1w_${sub}.job
done



for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/test.txt | tr '\n' ' ') ; do
sed -e "s|SUBJECT|${sub}|g" mriqc_rest_TEMPLATE.job > mriqc_rest_${sub}.job
done

for sub in $(cat /Shared/oleary/functional/UICL/BIDS/subject_list/test.txt | tr '\n' ' ') ; do
qsub mriqc_rest_${sub}.job
done

In [ ]:
singularity run -H /Users/tientong/singularity_home \
/Users/tientong/mriqc_0.15.1.sif \
/Shared/oleary/functional/UICL/BIDS/rawdata \
/Shared/oleary/functional/UICL/BIDS/derivatives/mriqc \
group

In [ ]:

singularity exec ~/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img 3dcalc \
-a /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/gen_ref/ref_bold.nii.gz \
-expr '(a)+500' \
-prefix /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/gen_ref/ref_bold500.nii.gz \
-datum float



singularity exec ~/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img \
N4BiasFieldCorrection \
--bspline-fitting [ 200 ] -d 3 \
--input-image /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/gen_ref/ref_bold500.nii.gz \
--mask-image /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/enhance_and_skullstrip_bold_wf/check_hdr/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask_trans_dil_hdr.nii.gz \
--output /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/enhance_and_skullstrip_bold_wf/n4_correct/ref_bold_corrected500.nii.gz



singularity exec ~/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img 3dcalc \
-a /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/enhance_and_skullstrip_bold_wf/n4_correct/ref_bold_corrected500.nii.gz \
-expr '(a)-500' \
-prefix /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest_time2/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/enhance_and_skullstrip_bold_wf/n4_correct/ref_bold_corrected.nii.gz \
-datum float

(base) [tientong@argon-login-1 n4_correct]$ singularity exec ~/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img fslstats /Users/tientong/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/gen_ref/ref_bold_500.nii.gz -R

483.276367 9947.896484 


(base) [tientong@argon-login-1 n4_correct]$ singularity exec ~/poldracklab_fmriprep_1.4.1-2019-07-09-86bf8bc4b7d5.img fslstats /Users/tientong/fmriprep_wf/single_subject_3203_wf/func_preproc_ses_61778918_task_rest_wf/bold_bold_trans_wf/bold_reference_wf/gen_ref/ref_bold.nii.gz -R

-16.723633 9447.896484



fslstats ref_bold_corrected.nii.gz -R

483.276367 9947.896484 

In [6]:
print(483.276367 - 500)
print(9947.896484 - 500)

-16.723633000000007
9447.896484


# Use the extended Power ROI 

Links: https://greenelab.wustl.edu/data_software

Downloaded to: /oleary/atlas/greeneatlas/orig

Then run 

* 01-create-coord.R
* 02-3dundump.sh

to create 300 spherical ROIs, or any other seeds of interest from this atlas

# Post-processing after fMRIPrep: Nuisance Regression & Signal extraction

Code in: /oleary/functional/UICL/BIDS/code/rest01_postfMRIPrep.sh

1. Non-aggressively denoise unsmooth BOLD with ICA-AROMA. 
2. Extract WM and CSF time series from the output of 1. 
    * this also called an R script to organize the confounds `rest02_makeconfound.R`
3. Nuisance regression and bandpass filter of the output from 2. 
4. Calculate correlation matrix of the 300 ROIs from the extended Power atlas.

In [1]:
%%bash
dir=/Shared/oleary/functional/UICL/BIDS/code
sh $dir/afterfMRIPrep_01runall.sh 2>&1 | tee $dir/afterfMRIPrep_01runall.sh.log


In [ ]:
3dresample \
  -master $fmriprepdir/sub-${subid}/ses-${sesid}/func/sub-${subid}_ses-${sesid}_task-rest_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz \
  -prefix /oleary/atlas/greeneatlas/greene300LPI.nii.gz \
  -input /oleary/atlas/greeneatlas/greene300.nii.gz

## Check if correlation matrices had been created for all subject

In [ ]:
%%bash

dir=/Shared/oleary/functional/UICL/BIDS/derivatives/subject_level_glm/rest

# time 1 only N = 102
ls $dir/time1only/sub-*/ses-*/sub-*-*_extendedPower_000.netcc > time1only.txt
# check against this file
gedit /Shared/oleary/functional/UICL/BIDS/subject_list/time1only.txt


# twovisitSiemens N = 21
ls $dir/twovisitSiemens/sub-*/ses-*/sub-*-*_extendedPower_000.netcc > twovisitSiemens.txt
# check against this file
gedit /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses_siemens.txt


# twovisitGE N = 98
ls $dir/twovisitGE/sub-*/ses-*/sub-*-*_extendedPower_000.netcc > twovisitGE.txt
# check against this file
gedit /Shared/oleary/functional/UICL/BIDS/code/time2/ge_time1time2.txt


##############################################################################

for type in time1only twovisitSiemens twovisitGE ; do
 cp $(cat $dir/${type}.txt | tr '\n' ' ') $dir/${type}/all_${type}/
 cd $dir/${type}
 zip -r all_${type}.zip all_${type}
done

# Seed-whole brain correlation analysis

Code: `/oleary/functional/UICL/BIDS/code/rest07_seedbased.sh`
    
* First, use `3dmaskave` to get time series of interested ROIs.
    *
    
* Then, use `3dTcorr1D` to get correlation between ROIs timeseries and the rest of the brain

* Lastly, use `3dcalc` to do Fisher's r to z transformation 

# Group analysis of seed based analysis: randomise

User guide here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Randomise/UserGuide

`randomise -i <4D_input_data> -o <output_rootname> -d design.mat -t design.con -m /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8_multicore/data/standard/MNI152_T1_2mm_brain_mask.nii.gz -n 5000 -T`

Tutorial here: https://www.youtube.com/watch?v=Ukl1VWobviw
Need: group average nifti, design matrix and contrast file

Code: `/oleary/functional/UICL/BIDS/code/randomise/rest` 

First, run `get_input.Rmd` to get list of files with subjects in the correct order

Then, run `fslmerge` with option `t` to concatnate all files to create the input of `randomise`. 

Lastly, run randomise scripts
    * Creat template file (pasted below) to run randomise across different ROIs and Contrasts
    * Run the for loop for specific ROI and Contrast

## Randomise: 40 non-bingeing controls vs. 118 bingers

In [ ]:
%%bash 
#/bin/bash
source ~/sourcefiles/fsl_source.sh

mask=/Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8_multicore/data/standard/MNI152_T1_2mm_brain_mask.nii.gz
outdir=/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest

randomise \
-i $outdir/input_ROI.nii.gz \
-o $outdir/ROI_CONTRAST \
-d CONTRAST.mat \
-t CONTRAST.con \
-m  $mask \
-n 5000 \
-T

In [ ]:
%%bash

for roi in LAmyg RAmyg LNAcc RNAcc ; do
    for contrast in ConvBinge ; do
        sed -e "s|ROI|${roi}|" -e "s|CONTRAST|${contrast}|" randomise_cov_TEMPLATE.sh > randomise_cov_${roi}_${contrast}.sh
    done
done

In [ ]:
%%bash

for roi in LAmyg RAmyg LNAcc RNAcc ; do
    for contrast in 5group ; do
        sed -e "s|ROI|${roi}|" -e "s|CONTRAST|${contrast}|" randomise_cov_TEMPLATE.sh > randomise_cov_${roi}_${contrast}.sh
        sed -i '$ s/$/ -f 5group_Cov.fts/' randomise_cov_${roi}_${contrast}.sh
    done
done

## Randomise 1 between-subject factor: Group (controls vs. sBinge vs. eBinge vs. sBinge+MJ vs. eBinge+MJ)

Have to add this flag in randomise command: `-f <design.fts> f contrasts file`. Read more [here](https://www.biac.duke.edu/forums/topic.asp?TOPIC_ID=1645)

## Repeat randomise adding covariates 

for both contrasts (5 group and ConvsBinge). The covariates are demeaned:

1. Original FD (prior to being ICA AROMA-denoised) - get this by running `/oleary/functional/UICL/BIDS/code/randomise/rest/getfd_rest.R`
2. Gender (Notes from FSL: Note that categorical covariates (e.g. gender) are treated in exactly the same way as continuous covariates - that is, use two indicator values (e.g. 0 and 1) and then demean these values before entering them into the EV.)
3. Recent monthly avg Tobacco

Notes from FSL: Age is mean centered in this model by subtracting the overall mean age from each individual age. Importantly, this is done across all subjects and not within group. The alternative, within-group mean centering, would remove any ability of age to describe group differences and if that was done the group difference inferences would not be adjusted for age. Consequently, the overall mean is removed here so that the inferences are adjusted for any age differences between the groups.

Viewing and Reporting Results
When viewing the 1-p results in FSLView the min/max display range should be set to 0.95/1.0 so that values less than 0.95 (equivalent to p>0.05) are not shown. If these are corrected values (i.e. corrp) then the visible areas correspond to the statistically significant regions.

To report cluster results (size, maxima, locations, labels) you can use the cluster and atlasquery tools.

Getting Cluster and Peak Information from Randomise Output
Assuming that the best image to get the clusters and peak information from is the raw tstat image:

begin by masking this with the significant voxels from corrp:

fslmaths grot_tfce_corrp_tstat1 -thr 0.95 -bin -mul grot_tstat1 grot_thresh_tstat1

Now run cluster to extract the clusters and local maxima in several different outputs:

cluster --in=grot_thresh_tstat1 --thresh=0.0001 --oindex=grot_cluster_index --olmax=grot_lmax.txt --osize=grot_cluster_size

If the data is already in standard space (MNI152) and co-ordinate reporting is wanted in MNI (mm) values, add --mm to the end of the command.

If clusters need to be forced to be more highly split, the --thresh value can be raised to an appropriate level. Because the input image was already masked by the corrp image, regardless of the thresh value used in the cluster command, only significant voxels will ever be reported.

### Results of 2 Group permutation with no covariate

In [7]:
%%bash
source /raid0/homes/tientong/sourcefiles/fsl_source.sh

#anything within the threshold
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest
for contrast in tstat1 tstat2 ; do
for roi in LAmyg RAmyg LNAcc RNAcc ; do
echo ${roi}_${contrast}
fslstats $dir/${roi}_ConvBinge_tfce_corrp_${contrast}.nii.gz -R
done
done


This version does not require a special version of Python. Moving on.
 
You should now be set up to run FSL v5.0.8_multicore
 
LAmyg_tstat1
0.000000 0.182000 
RAmyg_tstat1
0.000000 0.965000 
LNAcc_tstat1
0.000000 0.892400 
RNAcc_tstat1
0.000000 0.082600 
LAmyg_tstat2
0.000000 0.077400 
RAmyg_tstat2
0.000000 0.264400 
LNAcc_tstat2
0.000000 0.989200 
RNAcc_tstat2
0.000000 0.700600 


0.05 / 4 = 0.0125

1 - 0.0125 = 0.9875

### Results of 2 Group permutation with covariates

In [6]:
%%bash
source /raid0/homes/tientong/sourcefiles/fsl_source.sh

#anything within the threshold WITH COVARIATES
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate
for contrast in tstat1 tstat2 ; do
for roi in LAmyg RAmyg LNAcc RNAcc ; do
echo ${roi}_${contrast}
fslstats $dir/${roi}_ConvBinge_Cov_tfce_corrp_${contrast}.nii.gz -R
done
done

This version does not require a special version of Python. Moving on.
 
You should now be set up to run FSL v5.0.8_multicore
 
LAmyg_tstat1
0.000000 0.188000 
RAmyg_tstat1
0.000000 0.964000 
LNAcc_tstat1
0.000000 0.905400 
RNAcc_tstat1
0.000000 0.079000 
LAmyg_tstat2
0.000000 0.143800 
RAmyg_tstat2
0.000000 0.200800 
LNAcc_tstat2
0.000000 0.984200 
RNAcc_tstat2
0.000000 0.547400 


In [ ]:
%%bash
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate

# step 1: masking with the significant voxels from corrp
fslmaths \
$dir/RAmyg_ConvBinge_Cov_tfce_corrp_tstat1.nii.gz \
-thr 0.95 \
-bin \
-mul RAmyg_ConvBinge_Cov_tstat1.nii.gz \
$dir/RAmyg_Cov_Con-greater-Binge_thresh

fslmaths \
$dir/LNAcc_ConvBinge_Cov_tfce_corrp_tstat2.nii.gz \
-thr 0.95 \
-bin \
-mul LNAcc_ConvBinge_Cov_tstat2.nii.gz \
$dir/LNAcc_Cov_Binge-greater-Con_thresh

# run cluster to extract the clusters and local maxima in several different outputs:


/Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/6.0.3/bin/cluster \
--in=$dir/RAmyg_Cov_Con-greater-Binge_thresh \
--thresh=0.0001 --oindex=$dir/RAmyg_Cov_Con-greater-Binge_cluster_index \
--mm >> $dir/RAmyg_Cov_Con-greater-Binge.txt


/Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/6.0.3/bin/cluster \
--in=$dir/LNAcc_Cov_Binge-greater-Con_thresh \
--thresh=0.0001 --oindex=$dir/LNAcc_Cov_Binge-greater-Con_cluster_index \
--mm >> $dir/LNAcc_Cov_Binge-greater-Con.txt

### Results of 5 Group permutation with no covariate

In [1]:
%%bash
source /raid0/homes/tientong/sourcefiles/fsl_source.sh

#anything within the threshold
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest
for contrast in fstat1 ; do
for roi in LAmyg RAmyg LNAcc RNAcc ; do
echo ${roi}_${contrast}
fslstats $dir/${roi}_5group_tfce_corrp_${contrast}.nii.gz -R
done
done

This version does not require a special version of Python. Moving on.
 
You should now be set up to run FSL v5.0.8_multicore
 
LAmyg_fstat1
0.000000 0.569800 
RAmyg_fstat1
0.000000 0.912200 
LNAcc_fstat1
0.000000 0.977400 
RNAcc_fstat1
0.000000 0.623400 


### Results of 5 Group permutation with  covariates (monthly tobacco)

In [1]:
%%bash
source /raid0/homes/tientong/sourcefiles/fsl_source.sh
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate
for contrast in fstat1 ; do
for roi in LAmyg RAmyg LNAcc RNAcc ; do
echo ${roi}_${contrast}
fslstats $dir/${roi}_5group_Cov_tfce_corrp_${contrast}.nii.gz -R
done
done

This version does not require a special version of Python. Moving on.
 
You should now be set up to run FSL v5.0.8_multicore
 
LAmyg_fstat1
0.000000 0.560400 
RAmyg_fstat1
0.000000 0.897400 
LNAcc_fstat1
0.000000 0.971800 
RNAcc_fstat1
0.000000 0.568200 


In [ ]:
%%bash
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate

# step 1: masking with the significant voxels from corrp
fslmaths \
$dir/LNAcc_5group_Cov_tfce_corrp_fstat1.nii.gz \
-thr 0.95 \
-bin \
-mul $dir/LNAcc_5group_Cov_fstat1.nii.gz \
$dir/LNAcc_5group_Cov_thresh


# run cluster to extract the clusters and local maxima in several different outputs:

/Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/6.0.3/bin/cluster --in=$dir/LNAcc_5group_Cov_thresh \
--thresh=0.0001 --oindex=$dir/LNAcc_5group_Cov_cluster_index --mm >> $dir/LNAcc_5group_Cov.txt


In [ ]:
%%bash
cd /Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate
source /raid0/homes/tientong/sourcefiles/fsl_source.sh
source /raid0/homes/tientong/sourcefiles/afni_source.sh

fslview /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8_multicore/data/standard/MNI152_T1_2mm_brain.nii.gz RAmyg_ConvBinge_Cov_tfce_corrp_tstat1.nii.gz LNAcc_ConvBinge_Cov_tfce_corrp_tstat2.nii.gz LNAcc_5group_Cov_tfce_corrp_fstat1.nii.gz
afni /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8_multicore/data/standard/MNI152_T1_2mm_brain.nii.gz LNAcc_5group_Cov_cluster_index.nii.gz LNAcc_Cov_Binge-greater-Con_cluster_index.nii.gz RAmyg_Cov_Con-greater-Binge_cluster_index.nii.gz


In [ ]:
%%bash
dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate

/Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/6.0.3/bin/cluster \
-i $dir/LNAcc_5group_Cov_tfce_corrp_fstat1.nii.gz \
-t 0.95 -c $dir/LNAcc_5group_Cov_fstat1.nii.gz \
--oindex=$dir/LNAcc_5group_Cov_cluster_index  \
--scalarname="1-p" > LNAcc_5group_Cov.txt


In [ ]:
%%bash

dir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate

for cluster in 1 2 3 4 ; do

# step 1: making masks of significant clusters in LPI orientation
fslmaths \
$dir/LNAcc_5group_Cov_cluster_index.nii.gz \
-thr ${cluster} -uthr ${cluster} -bin \
$dir/LNAcc_5group_Cov_cluster_index-${cluster}.nii.gz


3dresample \
-input $dir/LNAcc_5group_Cov_cluster_index-${cluster}.nii.gz \
-orient LPI \
-prefix $dir/LNAcc_5group_Cov_cluster_index-${cluster}_LPI.nii.gz

done


In [3]:
%%bash

# step 2: extract bold data from those masks

#!/bin/bash

source ~/sourcefiles/fsl_source.sh
source ~/sourcefiles/afni_source.sh

bolddir=/Shared/oleary/functional/UICL/BIDS/derivatives/subject_level_glm/rest
maskdir=/Shared/oleary/functional/UICL/BIDS/derivatives/group_analysis/rest/covariate


sub=(3003 3004 3005 3006 3009 3011 3013 3016 3017 3018 3021 3022 3023 3024 3025 3026 3027 3029 3030 3031 3032 3034 3036 3037 3039 3040 3041 3042 3043 3044 3045 3046 3050 3051 3052 3055 3056 3057 3058 3059 3060 3061 3062 3063 3066 3067 3068 3069 3070 3071 3072 3073 3074 3075 3076 3077 3078 3079 3080 3081 3082 3083 3084 3085 3086 3087 3089 3091 3093 3094 3095 3096 3097 3098 3099 3100 3101 3102 3103 3104 3106 3107 3108 3110 3111 3112 3113 3114 3116 3117 3118 3120 3123 3125 3126 3127 3128 3129 3130 3132 3133 3134 3136 3137 3138 3139 3140 3141 3142 3144 3145 3148 3149 3150 3151 3152 3153 3155 3157 3158 3160 3161 3163 3164 3165 3166 3167 3168 3169 3171 3172 3173 3175 3176 3177 3181 3182 3183 3184 3185 3186 3187 3188 3189 3192 3193 3195 3197 3198 3199 3200 3201 3202 3203 3204 3205 3206 3207 3208 3209 4101 5000 5001 5002 5003 5004 5006 5007 5008 5009 5010 5011 5012 5014 5015 5016 5017 5018 5020 5022 5024 5025 5026 5027 5028 5029 5030 5031 5032 5034 5035 5036 5037 5038 5039 5040 5041 5043 5044 5047 5048 5050 5051 5052 5054 5055 5059 5061 5062 5064 5065 5066 5067 5068 5069 5070 5072 5073 5074 5075 5076)

ses=(60844114 60642114 60730114 61218714 60946214 60913814 61230414 61319014 61449114 61434514 61032114 61145414 61432614 61722814 61549914 61521914 61535914 61620714 61750814 61721914 61708114 61822714 61763914 63926214 62369814 64069714 64182514 64184914 64125314 64272114 64066514 64243014 64559714 64444714 64442614 64342014 64558514 64459014 64744514 64977014 64645914 64572314 64747014 65047514 64975114 64990514 64932914 65147714 65060914 65076014 60223115 65062714 60641315 65248914 60093215 60527515 60498815 60628415 60483615 60512015 60597115 60595615 60526115 60700715 60540315 60585615 60926915 61045415 60801315 60712915 60743615 60900715 60930815 61027015 61031615 61348315 60988915 61032515 61231215 61290115 61017415 61333815 61230315 61491515 61001215 61346915 61636915 61388015 61435015 61535415 61635515 61620715 61705915 61693915 61895515 61835415 61822415 61907115 62137915 63764115 63551315 63937815 63954715 63853815 63739515 63710315 63940415 64040415 63825815 63969115 64012615 64128115 64153315 64154215 64415115 63868015 64069515 64024015 64356315 64416115 64573115 64743515 64457015 64917015 64632215 64470115 64875115 65050315 64859015 64948715 64731715 64861115 65061515 60511116 60585716 60667716 60527716 60683216 60727216 60815816 60700916 60900316 60898816 61029516 61101116 61304416 61636716 61680116 61507016 61491816 61835516 61632616 61693416 61722516 61708616 61823616 61837616 61924316 61936216 61807916 64874513 63969514 64024314 64157414 64427014 64343414 64670814 64557614 64657514 64544814 64672514 64746014 64628814 64661014 64686414 64974114 65074914 60439215 60643215 60842315 61331615 61144615 61101215 60885915 61102415 61044415 60927915 61216515 61303315 61447415 61518315 61736215 61723415 61807915 63537915 61851815 63750515 61794415 63939515 64053115 64227215 64357315 64443515 64340315 64224815 65059715 64617215 64831215 64659615 64934915 64958315 60094116 60423316 60830516 61190016 61014516 61088816 61204616 61405416 61319516 61780216)

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 ; do

for cluster in 1 2 3 4 ; do

3dmaskave -quiet -mask $maskdir/LNAcc_5group_Cov_cluster_index-${cluster}_LPI.nii.gz \
          $bolddir/sub-${sub[$n]}/ses-${ses[$n]}/LNAcc_r2z.nii.gz \
          > $maskdir/LNAcc_5group_Cov/sub-${sub[$n]}_ses-${ses[$n]}_cluster-${cluster}_ts.txt

cat $maskdir/LNAcc_5group_Cov/sub-${sub[$n]}_ses-${ses[$n]}_cluster-${cluster}_ts.txt >> $maskdir/LNAcc_5group_Cov/cluster-${cluster}_ts.txt

done
done


This version does not require a special version of Python. Moving on.
 
You should now be set up to run FSL v5.0.8_multicore
 
 
You should now be set up to use AFNI v18.2.04


++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 4 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 39 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 39 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 49 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 4 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 39 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 39 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 49 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 4 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
+++ 39 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]


# xcpEngine - not use this right now

https://xcpengine.readthedocs.io/overview.html

now downloaded in /oleary/xcpEngine-master

install the python wrapper: pip install xcpengine-container

download singularity image using the following code: 

singularity build xcpEngine.simg docker://pennbbl/xcpengine:latest

In order to incorporate the new atlas to xcpEngine pipeline, need to follow the instructions in: https://xcpengine.readthedocs.io/development.html

1. Clonning the source code: git clone https://github.com/PennBBL/xcpEngine.git
2. Create a new folder in the atlas directory to include the new atlas
3. change the design file /xcpEngine-master/designs/fc-36p_scrub_mergedatlas.dsn so that all lines of codes that used power264 now used the merged atlas

# xcpEngine Step 1 - create a cohort file

https://xcpengine.readthedocs.io/config/cohort.html#cohortfile

In [ ]:
%%bash

ls /oleary/functional/UICL/BIDS/derivatives/fmriprep/rest/fmriprep/sub-*/ses-*/func/*rest*MNI152NLin2009cAsym_preproc.nii.gz > /oleary/functional/UICL/BIDS/derivatives/xcpengine/cohort01.csv

awk '{gsub("/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest/fmriprep/", "");print}' /oleary/functional/UICL/BIDS/derivatives/xcpengine/cohort01.csv > /oleary/functional/UICL/BIDS/derivatives/xcpengine/cohort02.csv


In [ ]:
# example script to run xcpengine on argon
# to understand the script, read this:
# https://sudoneuroscience.wordpress.com/2017/06/08/running-multiple-jobs-on-the-argon-cluster-example-with-fmriprep/



#!/bin/sh
#$ -pe smp 3
#$ -q PINC
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/xcpengine/out
#$ -e /Users/tientong/logs/uicl/xcpengine/err

singularity run \
  -B /Shared/oleary:/mnt \
  /Users/tientong/xcpEngine.simg \
  -d /Users/tientong/xcpEngine/designs/fc-36p_scrub.dsn \
  -c /mnt/functional/UICL/BIDS/derivatives/xcpengine/cohort.csv \
  -r /mnt/functional/UICL/BIDS/derivatives/fmriprep/rest/fmriprep \
  -i $TMPDIR \
  -o /mnt/functional/UICL/BIDS/derivatives/xcpengine

# Create an atlas

Download the following 3 atlases (now downloaded and are in **/oleary/atlas**):  
Buckner cerebellum: http://www.freesurfer.net/fswiki/CerebellumParcellation_Buckner2011  
Choi striatum: https://surfer.nmr.mgh.harvard.edu/fswiki/StriatumParcellation_Choi2012  
Schaefer cortical: https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal  

Use James's script **/oleary/atlas/merge_atlas.py** by running the line of code below, when you're in /oleary/atlas   

In [ ]:
%%bash
./merge_atlas.py \
-a Schaefer/MNI/Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii.gz \
Choi_JNeurophysiol12_MNI152/Choi2012_17Networks_MNI152_FreeSurferConformed1mm_LooseMask.nii.gz \
Buckner_JNeurophysiol11_MNI152/Buckner2011_17Networks_MNI152_FreeSurferConformed1mm_LooseMask.nii.gz \
-r /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.11_multicore/data/standard/MNI152_T1_2mm_brain.nii.gz \
-t schaefer_parcel-400_network-17.tsv -n Striatal Cerebellar

# Afterward, you'll have those files:  

# lut.tsv -- a look up table with the regions indexes and names    
# mergedAtlas.nii.gz -- a new atlas that are the combination of the 3 atlases

In [ ]:
#!/bin/sh

#$ -pe smp 2
#$ -q PINC
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/uicl/xcpengine/out
#$ -e /Users/tientong/logs/uicl/xcpengine/err

singularity run -B /Users/tientong/xcpEngine/atlas/merged:/xcpEngine/atlas/merged \
-H /Users/tientong/singularity_home \
/Users/tientong/xcpEngine.simg \
-d /Users/tientong/xcpEngine/designs/fc-aroma_mergedatlas.dsn \
-c /Shared/oleary/functional/UICL/BIDS/derivatives/xcpengine/cohort.csv \
-o /Shared/oleary/functional/UICL/BIDS/derivatives/xcpengine/testmergedatlas \
-t 1 -r /Shared/oleary/functional/UICL/BIDS/derivatives/fmriprep/rest/fmriprep

In [ ]:
%%bash
singularity shell \
  -B /Shared/oleary:/mnt \
  -B `pwd`/xcpEngine:/xcpEngine \
  xcpEngine.simg

In [ ]:
xcpEngine\
  -d /xcpEngine/designs/fc-36p_scrub_mergedatlas.dsn \
  -c /mnt/functional/UICL/BIDS/derivatives/xcpengine/cohort.csv \
  -i /mnt/functional/UICL/BIDS/derivatives/xcpengine \
  -o /mnt/functional/UICL/BIDS/derivatives/xcpengine